In [1]:
import numpy as np
import os
from DH_dynamics import dh_robot_config
from JS_control import *
import matplotlib.pyplot as plt
from pyrep import PyRep
from arm import CtRobot
from pyrep.backend import vrep
import sympy as sp
from pyrep.const import JointMode
import time

%matplotlib notebook

pr = PyRep()

In [2]:
param = ['D', 'a', 'alpha', 'theta', 'num_joints', 'jointType', 'Tbase', 'L', 'M']
config = dict()
for i in range(len(param)):
#     config[param[i]] = np.load('./robot_config/config_ct_7DOF/%s.npy'%param[i])
    config[param[i]] = np.load('./robot_config/inbore_config/%s.npy'%param[i])

In [3]:
robot = dh_robot_config(int(config['num_joints']), config['alpha'], config['theta'], config['D'], config['a'], 
                                                config['jointType'], config['Tbase'], config['L'], config['M'])
robot.initKinematicTransforms()

Calculating link 0
Calculating link 1
Calculating link 2
Calculating link 3
Calculating link 4
Calculating Mass and Gravity Matrix...
-------------------------------------------------------------------
Calculation complete


## Simulation Set up

### For all joints

In [4]:
pr.launch(os.getcwd() + '/ct_robot.ttt', headless=True)
ct_robot = CtRobot(name='inbore_arm', num_joints=4, joint_type=['r','r','r','p'])

xyz = np.array([0, 0, 0])

In [5]:
dt = 0.01
pr.set_simulation_timestep(dt)

In [9]:
# Starting simulation
pr.start()
err_pos = []
err_vel = []
mespos = []
depos = []

# Set up joints to be in force control mode
for i in range(ct_robot._num_joints):
    ct_robot.joints[i].set_joint_mode(JointMode.FORCE)
    ct_robot.joints[i].set_control_loop_enabled(False)
    ct_robot.joints[i].set_motor_locked_at_zero_velocity(True)
    ct_robot.joints[i].set_joint_position(0)
for j in range(1, ct_robot._num_joints + 1):
    ct_robot.arms[i].set_dynamic(True)
ct_robot.arms[0].set_dynamic(False)
pr.step()

# Desired trajectory
end_time = 4
t_dic = np.linspace(0, end_time, num=int(end_time/dt), endpoint=True)
t = sp.Symbol('t')
# disired trajectory of position for each joint

# # Sine wave activation
# traj = [
#     0.0*sp.sin(t*4),
#     0.0*sp.cos(t*4),
#     (0/180*np.pi)*sp.sin(t*np.pi),
#     (0/180*np.pi)*sp.sin(t*4),
#     (30/180*np.pi)*sp.cos(t*4),
#     (0/180*np.pi)*sp.cos(t*4),
#     0.00*sp.sin(t/2)
# ]

# Step response
traj = [
    (-30/180*np.pi)*sp.ones(1),
    (45/180*np.pi)*sp.ones(1),
    (20/180*np.pi)*sp.ones(1),
    0.005*sp.ones(1)
]

pos = [sp.lambdify(t, i) for i in traj]
vel = [sp.lambdify(t, i.diff(t)) for i in traj]
acc = [sp.lambdify(t, i.diff(t).diff(t)) for i in traj]

# Gain
kp = np.diag([1.2, 1, 0.4, 0.4])
# kv = np.diag([10, 7, 2, 3])
kv = 3*np.diag(kp)

# kp = np.eye(7)*10
# kv = 2*np.diag(kp)

# Simulation
for i in range(int(end_time/dt)):
    posd = [float(j(t_dic[i])) for j in pos]
    veld = [float(j(t_dic[i])) for j in vel]
    accd = [float(j(t_dic[i])) for j in acc]
    posm = ct_robot.get_joint_positions()
    velm = ct_robot.get_joint_velocities()
    ts1 = time.time()
    tau = cacl_tau_GravityCompensation(robot, kp, kv, posd, posm, veld, velm, xyz)
    te = time.time()
    print('Time for computing torque: %.6f'%(te-ts1))
    print(tau)
    
    ts2 = time.time()
    ct_robot.set_joint_target_velocities((np.sign(tau)*1000000).tolist())
    ct_robot.set_joint_forces(np.abs(tau).tolist())
    pr.step()
    measure_pos = np.array(ct_robot.get_joint_positions())
    measure_vel = np.array(ct_robot.get_joint_velocities())
    te1 = time.time()
    print('Time for communicating with V-REP: %.6f'%(te1-ts2))
    
    # Record error of each joint
    err_pos.append(posd - measure_pos)
    err_vel.append(veld - measure_vel)
    depos.append(posd)
    mespos.append(measure_pos)


Time for computing torque: 0.000497
[ 1.48818517e+01  3.41066323e+01 -2.27341164e-02  2.09264439e+01]
Time for communicating with V-REP: 0.001822
Time for computing torque: 0.000458
[-52.95583018  44.64523535  -0.60520383 -32.75062116]
Time for communicating with V-REP: 0.000204
Time for computing torque: 0.000466
[-52.95583018  44.64523535  -0.60520383 -32.75062116]
Time for communicating with V-REP: 0.000161
Time for computing torque: 0.000443
[-52.95583018  44.64523535  -0.60520383 -32.75062116]
Time for communicating with V-REP: 0.000228
Time for computing torque: 0.000442
[-52.95583018  44.64523535  -0.60520383 -32.75062116]
Time for communicating with V-REP: 0.000269
Time for computing torque: 0.000434
[-52.95583018  44.64523535  -0.60520383 -32.75062116]
Time for communicating with V-REP: 0.000189
Time for computing torque: 0.000433
[-52.95583018  44.64523535  -0.60520383 -32.75062116]
Time for communicating with V-REP: 0.000262
Time for computing torque: 0.000432
[-52.95583018 

Time for computing torque: 0.000440
[ 42.33796743  59.95756858 -18.24447735  -1.34103871]
Time for communicating with V-REP: 0.000414
Time for computing torque: 0.000476
[ 42.33796743  59.95756858 -18.24447735  -1.34103871]
Time for communicating with V-REP: 0.000147
Time for computing torque: 0.000443
[ 42.33796743  59.95756858 -18.24447735  -1.34103871]
Time for communicating with V-REP: 0.000205
Time for computing torque: 0.000544
[ 42.33796743  59.95756858 -18.24447735  -1.34103871]
Time for communicating with V-REP: 0.000284
Time for computing torque: 0.000445
[ 42.33796743  59.95756858 -18.24447735  -1.34103871]
Time for communicating with V-REP: 0.000249
Time for computing torque: 0.000447
[ 42.33796743  59.95756858 -18.24447735  -1.34103871]
Time for communicating with V-REP: 0.000220
Time for computing torque: 0.000470
[ 42.33796743  59.95756858 -18.24447735  -1.34103871]
Time for communicating with V-REP: 0.001986
Time for computing torque: 0.000444
[ 42.33796743  59.95756858

Time for communicating with V-REP: 0.007269


2000.0

In [ ]:
# Change Unit
err_pos = np.array(err_pos)
mespos = np.array(mespos)
depos = np.array(depos)

for name in ['err_pos', 'mespos', 'depos']:
    x = eval(name)
    x[:, 0:1] = x[:, 0:1]*1000
    x[:, 1:2] = x[:, 1:2]*1000
    x[:, 6:7] = x[:, 6:7]*1000
    x[:, 2:3] = x[:, 2:3]*180/np.pi
    x[:, 3:4] = x[:, 3:4]*180/np.pi
    x[:, 4:5] = x[:, 4:5]*180/np.pi
    x[:, 5:6] = x[:, 5:6]*180/np.pi

In [ ]:
# Plot error of position from joint to joint
plt.figure(figsize=(18,10))
plt.suptitle('Absoluter value')
for i in range(6):
    plt.subplot(2,4,i+1)
    plt.title('Joint%d'%(i+1))
    plt.plot(mespos[:, i], label='mesaured')
    plt.plot(depos[:, i], label='desired')
    plt.legend()
    if i in [0,1,6]:
        plt.ylabel('mm')
    else:
        plt.ylabel('deg')


In [ ]:
pr.stop()

In [ ]:
pr.shutdown()

In [ ]:
# Plot error of position from joint to joint
plt.figure(figsize=(18,10))
plt.suptitle('Error')
for i in range(6):
    plt.subplot(2,4,i+1)
    plt.title('Joint%d'%(i+1))
    plt.plot(np.array(err_pos)[:, i])
    if i in [0,1,6]:
        plt.ylabel('mm')
    else:
        plt.ylabel('deg')